In [1]:
from pyspark.sql.functions import *
d = spark.read.csv('/user/maxnet/database/sig.db/data_mac_res',sep='\x01')
d = d.withColumnRenamed('_c0','mac').withColumnRenamed('_c1','id')
d = d.withColumn('m1',substring(d.mac,1,1)).\
      withColumn('m2',substring(d.mac,2,1)).\
      withColumn('m3',substring(d.mac,3,1)).\
      withColumn('m4',substring(d.mac,4,1)).\
      withColumn('m5',substring(d.mac,5,1)).\
      withColumn('m6',substring(d.mac,6,1)).\
      withColumn('m7',substring(d.mac,7,1)).\
      withColumn('m8',substring(d.mac,8,1)).\
      withColumn('m9',substring(d.mac,9,1)).\
      withColumn('m10',substring(d.mac,10,1)).\
      withColumn('m11',substring(d.mac,11,1)).\
      withColumn('m12',substring(d.mac,12,1))  

In [2]:
d.show(truncate=False)

+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+
|mac         |id   |m1 |m2 |m3 |m4 |m5 |m6 |m7 |m8 |m9 |m10|m11|m12|
+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+
|B8C74A11FB4D|10533|B  |8  |C  |7  |4  |A  |1  |1  |F  |B  |4  |D  |
|D4503F4A9BEB|40209|D  |4  |5  |0  |3  |F  |4  |A  |9  |B  |E  |B  |
|14C697EA51C7|42209|1  |4  |C  |6  |9  |7  |E  |A  |5  |1  |C  |7  |
|386EA2DEC715|41158|3  |8  |6  |E  |A  |2  |D  |E  |C  |7  |1  |5  |
|40CD7A210159|48883|4  |0  |C  |D  |7  |A  |2  |1  |0  |1  |5  |9  |
|E4B2FBA9777E|39226|E  |4  |B  |2  |F  |B  |A  |9  |7  |7  |7  |E  |
|587A6AFB7819|42210|5  |8  |7  |A  |6  |A  |F  |B  |7  |8  |1  |9  |
|208B37F4F60A|10535|2  |0  |8  |B  |3  |7  |F  |4  |F  |6  |0  |A  |
|B0E5EDCCDF22|40780|B  |0  |E  |5  |E  |D  |C  |C  |D  |F  |2  |2  |
|C0F4E69F8772|10803|C  |0  |F  |4  |E  |6  |9  |F  |8  |7  |7  |2  |
|449EF91BE94D|39367|4  |4  |9  |E  |F  |9  |1  |B  |E  |9  |4  |D  |
|9C2EA1C7D9C1|39781|9  |C  |2  |E 

In [3]:
tmp = d.groupBy('id').count()
tmp = tmp.withColumnRenamed('count','c')

In [4]:
tmp.sort('c',ascending=True).show(truncate=False)

+-----+---+
|id   |c  |
+-----+---+
|12259|1  |
|12311|1  |
|12500|1  |
|39629|1  |
|11624|1  |
|12404|1  |
|10149|1  |
|41436|1  |
|12285|1  |
|10358|1  |
|12326|1  |
|10452|1  |
|12167|1  |
|12354|1  |
|12216|1  |
|12397|1  |
|12408|1  |
|12523|1  |
|12512|1  |
|12409|1  |
+-----+---+
only showing top 20 rows



In [5]:
tmp.select('c').summary().show()

+-------+------------------+
|summary|                 c|
+-------+------------------+
|  count|              4128|
|   mean| 49418.97747093023|
| stddev|228834.72165307103|
|    min|                 1|
|    25%|                91|
|    50%|               552|
|    75%|              4030|
|    max|           4435867|
+-------+------------------+



In [6]:
use = tmp.filter(tmp.c > 40000)

In [7]:
use.count(),use.distinct().count()

(470, 470)

In [8]:
use_df = use.repartition(1)
use_pandas = use_df.toPandas()

In [9]:
use_pandas.head()

,id,c
0,39005,539648
1,39492,769767
2,39512,851884
3,42208,1099617
4,41098,270884


In [10]:
use_id = list(use_pandas.id)

In [11]:
def filt(s):
    for x in use_id:
        if x in s:
            return 1
    else:
            return 0

In [12]:
ft = udf(lambda x:filt(x))

In [13]:
d_tmp = d.withColumn('filt',ft('id'))
df = d_tmp.filter(d_tmp.filt == 1).select('mac','id','m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12')

In [14]:
df.show(truncate=False)

+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+
|mac         |id   |m1 |m2 |m3 |m4 |m5 |m6 |m7 |m8 |m9 |m10|m11|m12|
+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+
|B8C74A11FB4D|10533|B  |8  |C  |7  |4  |A  |1  |1  |F  |B  |4  |D  |
|D4503F4A9BEB|40209|D  |4  |5  |0  |3  |F  |4  |A  |9  |B  |E  |B  |
|14C697EA51C7|42209|1  |4  |C  |6  |9  |7  |E  |A  |5  |1  |C  |7  |
|386EA2DEC715|41158|3  |8  |6  |E  |A  |2  |D  |E  |C  |7  |1  |5  |
|40CD7A210159|48883|4  |0  |C  |D  |7  |A  |2  |1  |0  |1  |5  |9  |
|E4B2FBA9777E|39226|E  |4  |B  |2  |F  |B  |A  |9  |7  |7  |7  |E  |
|587A6AFB7819|42210|5  |8  |7  |A  |6  |A  |F  |B  |7  |8  |1  |9  |
|208B37F4F60A|10535|2  |0  |8  |B  |3  |7  |F  |4  |F  |6  |0  |A  |
|B0E5EDCCDF22|40780|B  |0  |E  |5  |E  |D  |C  |C  |D  |F  |2  |2  |
|C0F4E69F8772|10803|C  |0  |F  |4  |E  |6  |9  |F  |8  |7  |7  |2  |
|449EF91BE94D|39367|4  |4  |9  |E  |F  |9  |1  |B  |E  |9  |4  |D  |
|9C2EA1C7D9C1|39781|9  |C  |2  |E 

In [15]:
df.select('id').distinct().count()

470

In [16]:
df.count(),df.distinct().count()

(194068115, 194068115)

In [17]:
df.dtypes

[('mac', 'string'),
 ('id', 'string'),
 ('m1', 'string'),
 ('m2', 'string'),
 ('m3', 'string'),
 ('m4', 'string'),
 ('m5', 'string'),
 ('m6', 'string'),
 ('m7', 'string'),
 ('m8', 'string'),
 ('m9', 'string'),
 ('m10', 'string'),
 ('m11', 'string'),
 ('m12', 'string')]

In [18]:
df.show(5,truncate=False)

+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+
|mac         |id   |m1 |m2 |m3 |m4 |m5 |m6 |m7 |m8 |m9 |m10|m11|m12|
+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+
|B8C74A11FB4D|10533|B  |8  |C  |7  |4  |A  |1  |1  |F  |B  |4  |D  |
|D4503F4A9BEB|40209|D  |4  |5  |0  |3  |F  |4  |A  |9  |B  |E  |B  |
|14C697EA51C7|42209|1  |4  |C  |6  |9  |7  |E  |A  |5  |1  |C  |7  |
|386EA2DEC715|41158|3  |8  |6  |E  |A  |2  |D  |E  |C  |7  |1  |5  |
|40CD7A210159|48883|4  |0  |C  |D  |7  |A  |2  |1  |0  |1  |5  |9  |
+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+
only showing top 5 rows



In [19]:
df = df.withColumn('f1',conv(df.m1, 16, 10)).withColumn('f2',conv(df.m2, 16, 10)).\
        withColumn('f3',conv(df.m3, 16, 10)).withColumn('f4',conv(df.m4, 16, 10)).\
        withColumn('f5',conv(df.m5, 16, 10)).withColumn('f6',conv(df.m6, 16, 10)).\
        withColumn('f7',conv(df.m7, 16, 10)).withColumn('f8',conv(df.m8, 16, 10)).\
        withColumn('f9',conv(df.m9, 16, 10)).withColumn('f10',conv(df.m10, 16, 10)).\
        withColumn('f11',conv(df.m11, 16, 10)).withColumn('f12',conv(df.m12, 16, 10))

In [20]:
df.show()

+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|         mac|   id| m1| m2| m3| m4| m5| m6| m7| m8| m9|m10|m11|m12| f1| f2| f3| f4| f5| f6| f7| f8| f9|f10|f11|f12|
+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|B8C74A11FB4D|10533|  B|  8|  C|  7|  4|  A|  1|  1|  F|  B|  4|  D| 11|  8| 12|  7|  4| 10|  1|  1| 15| 11|  4| 13|
|D4503F4A9BEB|40209|  D|  4|  5|  0|  3|  F|  4|  A|  9|  B|  E|  B| 13|  4|  5|  0|  3| 15|  4| 10|  9| 11| 14| 11|
|14C697EA51C7|42209|  1|  4|  C|  6|  9|  7|  E|  A|  5|  1|  C|  7|  1|  4| 12|  6|  9|  7| 14| 10|  5|  1| 12|  7|
|386EA2DEC715|41158|  3|  8|  6|  E|  A|  2|  D|  E|  C|  7|  1|  5|  3|  8|  6| 14| 10|  2| 13| 14| 12|  7|  1|  5|
|40CD7A210159|48883|  4|  0|  C|  D|  7|  A|  2|  1|  0|  1|  5|  9|  4|  0| 12| 13|  7| 10|  2|  1|  0|  1|  5|  9|
|E4B2FBA9777E|39226|  E|  4|  B|  2|  F|  B|  A|  9|  7|  7|  7|

In [21]:
df.dtypes

[('mac', 'string'),
 ('id', 'string'),
 ('m1', 'string'),
 ('m2', 'string'),
 ('m3', 'string'),
 ('m4', 'string'),
 ('m5', 'string'),
 ('m6', 'string'),
 ('m7', 'string'),
 ('m8', 'string'),
 ('m9', 'string'),
 ('m10', 'string'),
 ('m11', 'string'),
 ('m12', 'string'),
 ('f1', 'string'),
 ('f2', 'string'),
 ('f3', 'string'),
 ('f4', 'string'),
 ('f5', 'string'),
 ('f6', 'string'),
 ('f7', 'string'),
 ('f8', 'string'),
 ('f9', 'string'),
 ('f10', 'string'),
 ('f11', 'string'),
 ('f12', 'string')]

In [22]:
df = df.select('mac','id',col('f1').cast('float'),col('f2').cast('float')\
               ,col('f3').cast('float'),col('f4').cast('float')\
               ,col('f5').cast('float'),col('f6').cast('float')\
               ,col('f7').cast('float'),col('f8').cast('float')\
               ,col('f9').cast('float'),col('f10').cast('float')\
               ,col('f11').cast('float'),col('f12').cast('float'))

In [23]:
from pyspark.ml.feature import VectorAssembler
vec = VectorAssembler(inputCols=['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12'],outputCol='features')
new_df = vec.transform(df)

In [24]:
from pyspark.ml.feature import StringIndexer
si = StringIndexer(inputCol='id', outputCol='label')
si_model = si.fit(new_df)
df_final = si_model.transform(new_df)

In [25]:
df_final.show(5,truncate=False)

+------------+-----+----+---+----+----+----+----+----+----+----+----+----+----+-------------------------------------------------------+-----+
|mac         |id   |f1  |f2 |f3  |f4  |f5  |f6  |f7  |f8  |f9  |f10 |f11 |f12 |features                                               |label|
+------------+-----+----+---+----+----+----+----+----+----+----+----+----+----+-------------------------------------------------------+-----+
|B8C74A11FB4D|10533|11.0|8.0|12.0|7.0 |4.0 |10.0|1.0 |1.0 |15.0|11.0|4.0 |13.0|[11.0,8.0,12.0,7.0,4.0,10.0,1.0,1.0,15.0,11.0,4.0,13.0]|2.0  |
|D4503F4A9BEB|40209|13.0|4.0|5.0 |0.0 |3.0 |15.0|4.0 |10.0|9.0 |11.0|14.0|11.0|[13.0,4.0,5.0,0.0,3.0,15.0,4.0,10.0,9.0,11.0,14.0,11.0]|107.0|
|14C697EA51C7|42209|1.0 |4.0|12.0|6.0 |9.0 |7.0 |14.0|10.0|5.0 |1.0 |12.0|7.0 |[1.0,4.0,12.0,6.0,9.0,7.0,14.0,10.0,5.0,1.0,12.0,7.0]  |22.0 |
|386EA2DEC715|41158|3.0 |8.0|6.0 |14.0|10.0|2.0 |13.0|14.0|12.0|7.0 |1.0 |5.0 |[3.0,8.0,6.0,14.0,10.0,2.0,13.0,14.0,12.0,7.0,1.0,5.0] |1.0  |
|40CD7

In [26]:
df_final.dtypes

[('mac', 'string'),
 ('id', 'string'),
 ('f1', 'float'),
 ('f2', 'float'),
 ('f3', 'float'),
 ('f4', 'float'),
 ('f5', 'float'),
 ('f6', 'float'),
 ('f7', 'float'),
 ('f8', 'float'),
 ('f9', 'float'),
 ('f10', 'float'),
 ('f11', 'float'),
 ('f12', 'float'),
 ('features', 'vector'),
 ('label', 'double')]

In [27]:
trainDF,testDF = df_final.randomSplit([0.7,0.3],seed=0)

In [28]:
trainDF.count()

135846957

# Training

In [1]:
df_final = spark.read.parquet('/data/user/hive/warehouse/ian/feature/mac_features')
trainDF,testDF = df_final.randomSplit([0.7,0.3],seed=0)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
import numpy as np

rf = RandomForestClassifier()

paramGrid_rf = ParamGridBuilder()\
       .addGrid(rf.maxDepth,[3,4,5,6,7,8,9,10])\
       .addGrid(rf.numTrees,[10,15,20,25,30])\
       .build()

crossval_rf = CrossValidator(estimator=rf,
                            estimatorParamMaps=paramGrid_rf,
                            evaluator=MulticlassClassificationEvaluator(),
                            numFolds=5)

cvModel_rf = crossval_rf.fit(trainDF)

In [ ]:
best_model_rf = cvModel_rf.bestModel

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
my_eval_rf = BinaryClassificationEvaluator(
    rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_rf.evaluate(best_model_rf.transform(testDF))

# Apple

In [11]:
d1 = spark.read.csv('/user/maxnet/database/sig.db/term_desc_all',sep='\x01')
d1 = d1.withColumnRenamed('_c0','term_id')\
       .withColumnRenamed('_c1','brand')\
       .withColumnRenamed('_c2','cn_name')\
       .withColumnRenamed('_c3','en_name')\
       .withColumnRenamed('_c4','type1')\
       .withColumnRenamed('_c5','type2')\
       .withColumnRenamed('_c6','os')\
       .withColumnRenamed('_c7','dtime')\
       .withColumnRenamed('_c8','price')\
       .withColumnRenamed('_c9','remarks')\
       .withColumnRenamed('_c10','prio')
d1.show(5,truncate=False)

+-------+-----+--------------+--------------+--------+--------+-------+-------------+-----+-------+----+
|term_id|brand|cn_name       |en_name       |type1   |type2   |os     |dtime        |price|remarks|prio|
+-------+-----+--------------+--------------+--------+--------+-------+-------------+-----+-------+----+
|40262  |大显 |大显X6        |DAXIAN X6     |手机    |手机    |Android|null         |null |null   |4   |
|40979  |联想 |联想A2800d    |Lenovo A2800d |手机    |手机    |Android|2015/4/1 0:00|298.0|null   |4   |
|11401  |vivo |vivo Y91      |vivo Y91      |手机    |手机    |Android|null         |null |null   |4   |
|10980  |苹果 |iPad Pro 3(11)|iPad Pro 3(11)|平板    |平板    |IOS    |null         |null |null   |4   |
|11765  |长虹 |长虹F8        |CHANGHONG F8  |智能电视|智能电视|Android|null         |null |null   |4   |
+-------+-----+--------------+--------------+--------+--------+-------+-------------+-----+-------+----+
only showing top 5 rows



In [12]:
d2 = spark.read.csv('/user/maxnet/database/sig.db/term_oui',sep='\x01')
d2 = d2.withColumnRenamed('_c0','mac_6').withColumnRenamed('_c1','manu')
d2.show(5,truncate=False)

+------+--------------------------------+
|mac_6 |manu                            |
+------+--------------------------------+
|002272|american micro-fuel device corp.|
|00D0EF|IGT                             |
|086195|罗克韦尔                        |
|F4BD9E|思科                            |
|5885E9|锐尔觅                          |
+------+--------------------------------+
only showing top 5 rows



In [13]:
d1.count(),d2.count()

(4655, 33420)

In [14]:
d1.distinct().count(),d2.distinct().count()

(4655, 33420)

In [15]:
from pyspark.sql.functions import *
d = spark.read.csv('/user/maxnet/database/sig.db/data_mac_res',sep='\x01')
d = d.withColumnRenamed('_c0','mac').withColumnRenamed('_c1','id')
d = d.withColumn('m1_6',substring(d.mac,1,6))
d.show(5,truncate=False)

+------------+-----+------+
|mac         |id   |m1_6  |
+------------+-----+------+
|B8C74A11FB4D|10533|B8C74A|
|D4503F4A9BEB|40209|D4503F|
|14C697EA51C7|42209|14C697|
|386EA2DEC715|41158|386EA2|
|40CD7A210159|48883|40CD7A|
+------------+-----+------+
only showing top 5 rows



In [16]:
tmp_1 = d.join(d2,d.m1_6==d2.mac_6)
tmp_1.show(truncate=False)

+------------+-----+------+------+-------+
|mac         |id   |m1_6  |mac_6 |manu   |
+------------+-----+------+------+-------+
|B8C74A11FB4D|10533|B8C74A|B8C74A|OPPO   |
|D4503F4A9BEB|40209|D4503F|D4503F|OPPO   |
|14C697EA51C7|42209|14C697|14C697|OPPO   |
|386EA2DEC715|41158|386EA2|386EA2|vivo   |
|40CD7A210159|48883|40CD7A|40CD7A|海信   |
|E4B2FBA9777E|39226|E4B2FB|E4B2FB|苹果   |
|587A6AFB7819|42210|587A6A|587A6A|OPPO   |
|208B37F4F60A|10535|208B37|208B37|创维   |
|B0E5EDCCDF22|40780|B0E5ED|B0E5ED|华为   |
|C0F4E69F8772|10803|C0F4E6|C0F4E6|华为   |
|449EF91BE94D|39367|449EF9|449EF9|vivo   |
|9C2EA1C7D9C1|39781|9C2EA1|9C2EA1|小米   |
|FC1A11A7B1E1|41139|FC1A11|FC1A11|vivo   |
|EC26CA7EAF23|50485|EC26CA|EC26CA|TP-LINK|
|90F052687CD0|42240|90F052|90F052|魅族   |
|A89CED8930DE|11107|A89CED|A89CED|小米   |
|3412F91C975A|39525|3412F9|3412F9|华为   |
|7836CCC9B4A5|42212|7836CC|7836CC|OPPO   |
|2C5BB8C2A126|41330|2C5BB8|2C5BB8|OPPO   |
|9C0CDFE5D2AB|10858|9C0CDF|9C0CDF|OPPO   |
+------------+-----+------+--

In [17]:
df = tmp_1.join(d1,tmp_1.id == d1.term_id).select('mac','id','manu','brand','type1','type2','cn_name').distinct().dropna()
df.count()

202742248

In [18]:
df.show(10,truncate=False)

+------------+-----+---------------------------+--------+------------+--------+-----------------+
|mac         |id   |manu                       |brand   |type1       |type2   |cn_name          |
+------------+-----+---------------------------+--------+------------+--------+-----------------+
|000000EC9011|39105|施乐                       |小米    |平板        |平板    |小米平板3        |
|000088000C1C|10550|博科                       |多度    |智能门禁    |智能家居|多度门禁         |
|0000C001FB25|11071|western                    |西部数据|家庭网络存储|智能家居|西部数据My Cloud |
|0006A9DE0D53|50485|universal instruments corp.|TP-LINK |路由器      |路由器  |TP-LINK TL-WR886N|
|00082204E9A5|40231|华宝通信                   |小米    |手机        |手机    |小米红米Note 4   |
|00082212D1E7|41009|华宝通信                   |魅族    |手机        |手机    |魅族MX5          |
|000822260D43|39536|华宝通信                   |荣耀    |手机        |手机    |荣耀畅玩5C       |
|00082226C3BD|41018|华宝通信                   |魅族    |手机        |手机    |魅族MX6          |
|000822387451|41024|华宝通信  

In [19]:
df.groupBy('manu').count().sort('count',ascending=False).show(truncate=False)

+-------+--------+
|manu   |count   |
+-------+--------+
|华为   |51105685|
|vivo   |48996433|
|OPPO   |26322420|
|小米   |21645498|
|TP-LINK|13737598|
|苹果   |12158969|
|水星   |2523718 |
|腾达   |2150727 |
|闻泰   |1929935 |
|迅捷   |1716830 |
|村田   |1654132 |
|魅族   |1613770 |
|金立   |1603072 |
|三星   |1438685 |
|蜂联   |904951  |
|风寻   |766455  |
|创维   |542278  |
|华三   |538901  |
|中兴   |537823  |
|天猫   |535237  |
+-------+--------+
only showing top 20 rows



In [20]:
df.groupBy('brand').count().sort('count',ascending=False).show(truncate=False)

+-------+--------+
|brand  |count   |
+-------+--------+
|vivo   |49502860|
|华为   |29187478|
|OPPO   |25592710|
|小米   |22934044|
|荣耀   |22128130|
|TP-LINK|12815942|
|苹果   |12098330|
|魅族   |4916567 |
|三星   |3152974 |
|水星   |3016162 |
|腾达   |2339608 |
|迅捷   |2096553 |
|金立   |1651077 |
|360    |1302378 |
|天猫   |902182  |
|HTC    |796477  |
|海信   |712247  |
|创维   |576193  |
|华三   |525402  |
|一加   |483060  |
+-------+--------+
only showing top 20 rows



In [21]:
df.select('manu').distinct().count(),df.select('brand').distinct().count()

(9789, 297)

In [25]:
apple = df.filter(df.brand == '苹果')

In [26]:
apple.count(),apple.distinct().count()

(12098330, 12098330)

In [27]:
apple.show(truncate=False)

+------------+-----+----+-----+-----+-----+--------------+
|mac         |id   |manu|brand|type1|type2|cn_name       |
+------------+-----+----+-----+-----+-----+--------------+
|CC4463F00249|39149|苹果|苹果 |平板 |平板 |iPad Pro(12.9)|
|64B0A6B9B4CE|49814|苹果|苹果 |手机 |手机 |iPhone 7 Plus |
|D461DA2F8EE3|39147|苹果|苹果 |手机 |手机 |iPhone 8 Plus |
|14BD61AF66A6|49814|苹果|苹果 |手机 |手机 |iPhone 7 Plus |
|88190820324D|39147|苹果|苹果 |手机 |手机 |iPhone 8 Plus |
|844167560490|49814|苹果|苹果 |手机 |手机 |iPhone 7 Plus |
|94F6D6A4DDC9|39000|苹果|苹果 |手机 |手机 |iPhone 8      |
|98CA3321A50C|49813|苹果|苹果 |手机 |手机 |iPhone 7      |
|741BB2A432CA|49807|苹果|苹果 |手机 |手机 |iPhone 6      |
|404D7FB8537E|49813|苹果|苹果 |手机 |手机 |iPhone 7      |
|7C0191DFBF7C|49810|苹果|苹果 |手机 |手机 |iPhone 6s Plus|
|E0B52D605ECF|49806|苹果|苹果 |手机 |手机 |iPhone 5s     |
|BC4CC4F29DC9|49808|苹果|苹果 |手机 |手机 |iPhone 6 Plus |
|1040F3B7DE56|49852|苹果|苹果 |平板 |平板 |iPad 3        |
|E49A792A2884|49809|苹果|苹果 |手机 |手机 |iPhone 6s     |
|38CADADDBBFC|49809|苹果|苹果 |手机 |手机 |iPhone 6s     |
|18F1D8

In [28]:
apple.select('id').distinct().count(),apple.select('cn_name').distinct().count()

(52, 52)

In [30]:
apple = apple.withColumn('m7',substring(d.mac,7,1))\
             .withColumn('m8',substring(d.mac,8,1))\
             .withColumn('m9',substring(d.mac,9,1))\
             .withColumn('m10',substring(d.mac,10,1))\
             .withColumn('m11',substring(d.mac,11,1))\
             .withColumn('m12',substring(d.mac,12,1))  

In [31]:
apple.show(5,truncate=False)

+------------+-----+----+-----+-----+-----+--------------+---+---+---+---+---+---+
|mac         |id   |manu|brand|type1|type2|cn_name       |m7 |m8 |m9 |m10|m11|m12|
+------------+-----+----+-----+-----+-----+--------------+---+---+---+---+---+---+
|68FB7EEDEA6F|49810|苹果|苹果 |手机 |手机 |iPhone 6s Plus|E  |D  |E  |A  |6  |F  |
|D033113FD082|49807|苹果|苹果 |手机 |手机 |iPhone 6      |3  |F  |D  |0  |8  |2  |
|B8C111BAF028|39226|苹果|苹果 |手机 |手机 |iPhone X      |B  |A  |F  |0  |2  |8  |
|9CF48EB8E028|49809|苹果|苹果 |手机 |手机 |iPhone 6s     |B  |8  |E  |0  |2  |8  |
|38892C701E18|39226|苹果|苹果 |手机 |手机 |iPhone X      |7  |0  |1  |E  |1  |8  |
+------------+-----+----+-----+-----+-----+--------------+---+---+---+---+---+---+
only showing top 5 rows



In [33]:
apple.groupBy('cn_name').count().sort('count',ascending=True).show(truncate=False)

+------------------------+-----+
|cn_name                 |count|
+------------------------+-----+
|iMac Pro                |4    |
|Mac Pro                 |25   |
|Mac Mini                |53   |
|iPad 1                  |1201 |
|苹果AirPort Time Capsule|1824 |
|苹果Airport Extreme     |2690 |
|苹果HomePod             |2724 |
|苹果AirPort Express     |2737 |
|iMac                    |2965 |
|iPad Pro 3(12.9)        |4553 |
|苹果电视机顶盒          |5615 |
|iPad Pro 2(12.9)        |5749 |
|iPad Air 3              |6288 |
|iPad Mini 5             |8524 |
|iPad Pro(12.9)          |9136 |
|iPad Pro 3(11)          |15703|
|iPod                    |18520|
|iPhone 5c               |20997|
|iPhone 4                |23975|
|iPad Mini 3             |24470|
+------------------------+-----+
only showing top 20 rows



In [35]:
apple.groupBy('cn_name').count().sort('count',ascending=False).show(truncate=True)

+--------------+-------+
|       cn_name|  count|
+--------------+-------+
|      iPhone 6|1210310|
|      iPhone X|1152331|
| iPhone 7 Plus|1134005|
|iPhone 6s Plus|1071258|
|     iPhone 6s|1031879|
|      iPhone 7| 923364|
| iPhone 6 Plus| 736422|
| iPhone 8 Plus| 705292|
|      iPhone 8| 500997|
|     iPhone XR| 411806|
| iPhone XS Max| 396816|
|      苹果手表| 337259|
|     iPhone 5s| 301883|
|    iPad Air 2| 270244|
|iPad 9.7(2018)| 225769|
|      iPad Air| 182333|
|    iPad (9.7)| 158510|
|   iPad Mini 2| 156222|
|   iPad Mini 4| 152068|
|     iPad Mini| 134536|
+--------------+-------+
only showing top 20 rows



In [36]:
tmp = apple.groupBy('cn_name').count().withColumnRenamed('count','c')
tmp.select('c').summary().show()

+-------+-----------------+
|summary|                c|
+-------+-----------------+
|  count|               52|
|   mean|232660.1923076923|
| stddev|355764.6767980998|
|    min|                4|
|    25%|             6288|
|    50%|            62143|
|    75%|           270244|
|    max|          1210310|
+-------+-----------------+



In [37]:
use = tmp.filter(tmp.c > 200000)

In [38]:
use_df = use.repartition(1)
use_pandas = use_df.toPandas()

In [39]:
use_pandas.head()

,cn_name,c
0,iPhone 7 Plus,1134005
1,iPhone XR,411806
2,iPhone 6s Plus,1071258
3,iPhone 6 Plus,736422
4,iPhone 6s,1031879


In [40]:
use_id = list(use_pandas.cn_name)
def filt(s):
    for x in use_id:
        if x in s:
            return 1
    else:
            return 0
ft = udf(lambda x:filt(x))
apple_tmp = apple.withColumn('filt',ft('cn_name'))
apple_final = apple_tmp.filter(apple_tmp.filt == 1).select('mac','id','cn_name','m7','m8','m9','m10','m11','m12')

In [41]:
apple_final.select('id').distinct().count()

16

In [42]:
apple_final.select('cn_name').distinct().count()

16

In [43]:
apple_final.show(5,truncate=False)

+------------+-----+--------------+---+---+---+---+---+---+
|mac         |id   |cn_name       |m7 |m8 |m9 |m10|m11|m12|
+------------+-----+--------------+---+---+---+---+---+---+
|48A1954188E9|49814|iPhone 7 Plus |4  |1  |8  |8  |E  |9  |
|6CAB318B43A2|49810|iPhone 6s Plus|8  |B  |4  |3  |A  |2  |
|F431C3BB37C0|49809|iPhone 6s     |B  |B  |3  |7  |C  |0  |
|507A55E6DCB1|49808|iPhone 6 Plus |E  |6  |D  |C  |B  |1  |
|E4B2FB0ED495|10950|iPhone XR     |0  |E  |D  |4  |9  |5  |
+------------+-----+--------------+---+---+---+---+---+---+
only showing top 5 rows



In [45]:
apple_final = apple_final.withColumn('f7',conv(apple_final.m7, 16, 10)).withColumn('f8',conv(apple_final.m8, 16, 10))\
                         .withColumn('f9',conv(apple_final.m9, 16, 10)).withColumn('f10',conv(apple_final.m10, 16, 10))\
                         .withColumn('f11',conv(apple_final.m11, 16, 10)).withColumn('f12',conv(apple_final.m12, 16, 10))

In [48]:
apple_final.show(5,truncate=False)

+------------+-----+--------------+---+---+---+---+---+---+---+---+---+---+---+---+
|mac         |id   |cn_name       |m7 |m8 |m9 |m10|m11|m12|f7 |f8 |f9 |f10|f11|f12|
+------------+-----+--------------+---+---+---+---+---+---+---+---+---+---+---+---+
|28FF3C0B189D|39147|iPhone 8 Plus |0  |B  |1  |8  |9  |D  |0  |11 |1  |8  |9  |13 |
|BCB8635EB40D|39890|iPad 9.7(2018)|5  |E  |B  |4  |0  |D  |5  |14 |11 |4  |0  |13 |
|90B0ED70F6CB|49810|iPhone 6s Plus|7  |0  |F  |6  |C  |B  |7  |0  |15 |6  |12 |11 |
|40831D9248BE|49814|iPhone 7 Plus |9  |2  |4  |8  |B  |E  |9  |2  |4  |8  |11 |14 |
|28FF3C145DC3|39147|iPhone 8 Plus |1  |4  |5  |D  |C  |3  |1  |4  |5  |13 |12 |3  |
+------------+-----+--------------+---+---+---+---+---+---+---+---+---+---+---+---+
only showing top 5 rows



In [50]:
apple_final = apple_final.select('mac','id','cn_name',\
                                 col('f7').cast('float'),col('f8').cast('float'),\
                                 col('f9').cast('float'),col('f10').cast('float'),\
                                 col('f11').cast('float'),col('f12').cast('float'))

In [51]:
from pyspark.ml.feature import VectorAssembler
vec = VectorAssembler(inputCols=['f7','f8','f9','f10','f11','f12'],outputCol='features')
new_df = vec.transform(apple_final)

In [52]:
from pyspark.ml.feature import StringIndexer
si = StringIndexer(inputCol='id', outputCol='label')
si_model = si.fit(new_df)
df_final = si_model.transform(new_df)

In [53]:
df_final.dtypes

[('mac', 'string'),
 ('id', 'string'),
 ('cn_name', 'string'),
 ('f7', 'float'),
 ('f8', 'float'),
 ('f9', 'float'),
 ('f10', 'float'),
 ('f11', 'float'),
 ('f12', 'float'),
 ('features', 'vector'),
 ('label', 'double')]

In [54]:
df_final.show(5,truncate=False)

+------------+-----+-------------+----+----+----+----+----+----+----------------------------+-----+
|mac         |id   |cn_name      |f7  |f8  |f9  |f10 |f11 |f12 |features                    |label|
+------------+-----+-------------+----+----+----+----+----+----+----------------------------+-----+
|54AE27C7C834|49806|iPhone 5s    |12.0|7.0 |12.0|8.0 |3.0 |4.0 |[12.0,7.0,12.0,8.0,3.0,4.0] |12.0 |
|F83880B8C56D|10680|iPhone XS Max|11.0|8.0 |12.0|5.0 |6.0 |13.0|[11.0,8.0,12.0,5.0,6.0,13.0]|10.0 |
|18F1D8CF0D01|39147|iPhone 8 Plus|12.0|15.0|0.0 |13.0|0.0 |1.0 |[12.0,15.0,0.0,13.0,0.0,1.0]|7.0  |
|20AB37888992|49813|iPhone 7     |8.0 |8.0 |8.0 |9.0 |9.0 |2.0 |[8.0,8.0,8.0,9.0,9.0,2.0]   |5.0  |
|B48B195A39D2|49809|iPhone 6s    |5.0 |10.0|3.0 |9.0 |13.0|2.0 |[5.0,10.0,3.0,9.0,13.0,2.0] |4.0  |
+------------+-----+-------------+----+----+----+----+----+----+----------------------------+-----+
only showing top 5 rows



In [55]:
df_final.select('label').distinct().count()

16

In [56]:
df_final.groupBy('id').count().show()

+-----+-------+
|   id|  count|
+-----+-------+
|49810|1071258|
|39890| 225769|
|49809|1031879|
|49866| 270244|
|49880| 337259|
|39147| 705292|
|49807|1210310|
|39000| 500997|
|49814|1134005|
|49813| 923364|
|39226|1152331|
|10679| 103451|
|10950| 411806|
|10680| 396816|
|49806| 301883|
|49808| 736422|
+-----+-------+



In [57]:
fractions = df_final.select('id').distinct().withColumn('fraction', lit(0.7)).rdd.collectAsMap()

In [58]:
trainDF = df_final.stat.sampleBy('id', fractions, seed=12)

In [59]:
trainDF.count()

7360274

In [60]:
trainDF.groupBy('id').count().show()

+-----+------+
|   id| count|
+-----+------+
|49810|750659|
|39890|158369|
|49809|722750|
|49866|189361|
|49880|236087|
|39147|494549|
|49807|846007|
|39000|350370|
|49814|794252|
|49813|646746|
|39226|806888|
|10679| 72025|
|10950|288316|
|10680|277375|
|49806|211481|
|49808|515039|
+-----+------+



In [61]:
train_mac = trainDF.select('mac').withColumnRenamed('mac','train_mac')
tmp = df_final.join(train_mac,df_final.mac==train_mac.train_mac,how='outer')
testDF = tmp.filter(tmp.train_mac.isNull())

In [62]:
testDF.count()

3152812

In [63]:
testDF = testDF.drop('train_mac')

In [64]:
testDF.groupBy('id').count().show()

+-----+------+
|   id| count|
+-----+------+
|49810|320964|
|39890| 67298|
|49809|309915|
|49866| 80547|
|49880|100893|
|39147|211553|
|49807|362566|
|39000|150962|
|49814|339746|
|49813|277206|
|39226|346070|
|10679| 31331|
|10950|123027|
|10680|119170|
|49806| 90346|
|49808|221218|
+-----+------+



In [65]:
df_final.groupBy('id').count().show()

+-----+-------+
|   id|  count|
+-----+-------+
|49810|1071258|
|39890| 225769|
|49809|1031879|
|49866| 270244|
|49880| 337259|
|39147| 705292|
|49807|1210310|
|39000| 500997|
|49814|1134005|
|49813| 923364|
|39226|1152331|
|10679| 103451|
|10950| 411806|
|10680| 396816|
|49806| 301883|
|49808| 736422|
+-----+-------+



In [68]:
#trainDF.write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/feature/trainDF',compression='gzip')

In [69]:
#testDF.write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/feature/testDF',compression='gzip')

In [71]:
#df_final.write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/feature/all_features',compression='gzip')